In [ ]:
#  import T5
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config
import numpy as np
import os

import torch
from datasets import load_dataset
from transformers import (
    RobertaTokenizerFast,
    RobertaForSequenceClassification,
    TrainingArguments,
    Trainer,
    DebertaTokenizer,
    DebertaForSequenceClassification,
    AutoConfig,
)

import logging
import evaluate
from evaluate import load
from datasets import load_dataset

from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import T5Tokenizer,T5ForConditionalGeneration

import nltk
from nltk.translate.bleu_score import corpus_bleu

import matplotlib.pyplot as plt

from  transformers  import  AutoTokenizer, AutoModelWithLMHead

import csv
from torch.utils.data import DataLoader, Dataset
import json
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
import pickle
import numpy as np
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

device=""
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f'Using GPU: {torch.cuda.get_device_name()}')
else:
    device = torch.device("cpu")
    print('Using CPU')

In [ ]:
PATH="/kaggle/input/webis-clickbait-22/"
OUTPATH="/kaggle/working/"
# PATH="D:\\ghd\\NLP-Project\\webis-clickbait-22\\"
# OUTPATH="D:\\ghd\\NLP-Project\\webis-clickbait-22\\output\\"
BATCH=8
T5_model="/kaggle/input/nlpprojt5/T5_overall"#phrase
T5_model="/kaggle/input/nlpprojt5/T5_overall_passage"
T5_model="/kaggle/input/nlpprojt5/T5_overall_multi"
sep='[SEP]'

In [ ]:
tag="phrase"

In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
def remove_escape(l):
    return l.replace("\n",". ")

In [ ]:
def get_label(x):
    return " ".join(x)

In [ ]:
train_data=pd.read_json("/kaggle/input/retriever/final_data_train.json")
val_data=pd.read_json("/kaggle/input/retriever/final_data_validation.json")
test_data=pd.read_json("/kaggle/input/retriever/final_data_test.json")

train_data = train_data.replace({None: ''})
val_data = val_data.replace({None: ''})
test_data = test_data.replace({None: ''})

train_real=pd.read_json("/kaggle/input/webis-clickbait-22/train.jsonl",lines=True)
val_real=pd.read_json("/kaggle/input/webis-clickbait-22/validation.jsonl",lines=True)
test_real=pd.read_json("/kaggle/input/webis-clickbait-22/test.jsonl",lines=True)

train_labels=train_real['spoiler'].apply(get_label)
val_labels=val_real['spoiler'].apply(get_label)
test_labels=test_real['spoiler'].apply(get_label)

train_tags=train_real['tags'].apply(get_label)
val_tags=val_real['tags'].apply(get_label)
test_tags=test_real['tags'].apply(get_label)
# train_type=train_real['spoiler']
train_data['exctractedParagraph']=train_data['exctractedParagraph'].apply(remove_escape)
val_data['exctractedParagraph']=val_data['exctractedParagraph'].apply(remove_escape)
test_data['exctractedParagraph']=test_data['exctractedParagraph'].apply(remove_escape)

train_data['context']=train_data['targetTitle'] + ' ' + train_data['targetDescription']+" "+train_data['exctractedParagraph']
val_data['context']=val_data['targetTitle'] + ' ' + val_data['targetDescription']+" "+val_data['exctractedParagraph']
test_data['context']=test_data['targetTitle'] + ' ' + test_data['targetDescription']+" "+test_data['exctractedParagraph']

train_data['question']=train_data['postText']+"?"
val_data['question']=val_data['postText']+'?'
test_data['question']=test_data['postText']+"?"

train_data['spoiler']=train_labels
val_data['spoiler']=val_labels
test_data['spoiler']=test_labels

train_data['tags']=train_tags
val_data['tags']=val_tags
test_data['tags']=test_tags

train_data=train_data[train_data['tags']==tag]
val_data=val_data[val_data['tags']==tag]
test_data=test_data[test_data['tags']==tag]

train_data=train_data[['question','context','spoiler']]

val_data=val_data[['question','context','spoiler']]
test_data=test_data[['question','context','spoiler']]


In [ ]:
train_data

In [ ]:
def preproc(data):
    q=[]
    c=[]
    a=[]
    # print(len(data))
    # print(len(data['question']))
    # print(len(data['context']))
    # print(len(data['spoiler']))
    for i in range(len(data['question'])):
        q.append(data['question'][i])
        c.append(data['context'][i])
        if c[i]==None:
            c[i]=""
        a.append(str(data['spoiler'][i]))
    model_inputs=tokenizer(q,c,text_target=a,return_tensors='pt',padding=True,truncation=True,max_length=512)
    return model_inputs

In [ ]:
# train_data=Dataset.from_pandas(train_df)
tokenized_train=train_dataset.map(preproc,batched=True,batch_size=BATCH)
tokenized_val=val_dataset.map(preproc,batched=True,batch_size=BATCH)
tokenized_test=test_dataset.map(preproc,batched=True,batch_size=BATCH)
tokenized_train,tokenized_val,tokenized_test

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=T5_model)

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained(T5_model)
model.to(device)

In [ ]:
bertscore = load("bertscore")
meteor = evaluate.load("meteor")
bleu = evaluate.load("bleu")

In [ ]:

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    pred, ref = postprocess_text(decoded_preds, decoded_labels)

    # cal bleu, meteor and bert
    bleu_score = bleu.compute(predictions=pred, references=ref)
#     bleu1 = corpus_bleu(ref,pred,weights=(1,0,0,0))
#     bleu2 = corpus_bleu(ref,pred,weights=(0.5,0.5,0))
#     bleu3 = corpus_bleu(ref,pred,weights=(0.33,0.33,0.33,0)
#     bleu4 = corpus_bleu(ref,pred,weights=(0.25,0.25,0.25,0.25))

    print(bleu_score)
    meteor_score = meteor.compute(predictions=pred, references=ref)
    bertscore_score = bertscore.compute(predictions=pred, references=ref, lang="en")

    #  dict
    return {"blue":bleu_score["bleu"],
            "precisions_1":bleu_score["precisions"][0],
            "precisions_2":bleu_score["precisions"][1],
            "precisions_3":bleu_score["precisions"][2],
            "precisions_4":bleu_score["precisions"][3],
            "bp":bleu_score["brevity_penalty"],
            "meteor": meteor_score["meteor"], 
            "bertscore_f1": np.average(bertscore_score["f1"]), 
            "bertscore_p": np.average(bertscore_score["precision"]), 
            "bertscore_r": np.average(bertscore_score["recall"])}


In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./T5",
    evaluation_strategy="epoch",
    learning_rate=2e-4,
    per_device_train_batch_size=BATCH,
    per_device_eval_batch_size=BATCH,
    weight_decay=0.01,
    save_total_limit=5,
    num_train_epochs=12,
    predict_with_generate=True,
    save_strategy="epoch",
    fp16=True,
#     report_to="wandb",
#     logging_dir='./lolololol'

    # push_to_hub=True
    # load_best_model_at_end=True,
    # metric_for_best_model="bleu"
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# trainer.train()

In [ ]:
def calculate_bleu(bp,precisions):
    weights_1 = np.array([1,0,0,0])
    weights_2 = np.array([0.5,0.5,0,0])
    weights_3 = np.array([0.33,0.33,0.33,0])
    weights_4 = np.array([0.25,0.25,0.25,0.25])
    logp = np.log(precisions)
        
    res1 = bp*np.exp(np.dot(logp,weights_1))
    res2 = bp*np.exp(np.dot(logp,weights_2))
    res3 = bp*np.exp(np.dot(logp,weights_3))
    res4 = bp*np.exp(np.dot(logp,weights_4))
    
    return [res1,res2,res3,res4]



In [ ]:
# results=trainer.evaluate()

In [ ]:
# calculate_bleu(results['eval_bp'],[results['eval_precisions_1'],results['eval_precisions_2'],results['eval_precisions_3'],results['eval_precisions_4']])

In [ ]:
results = trainer.evaluate(eval_dataset=tokenized_test)
print(results)

In [ ]:
calculate_bleu(results['eval_bp'],[results['eval_precisions_1'],results['eval_precisions_2'],results['eval_precisions_3'],results['eval_precisions_4']])